In [132]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles
using Random

In [1]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [134]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity30_.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance30.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"),header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File(string(path,"D_Dsoc.csv"),header=1,delim=",") |> DataFrame
emi_sink = CSV.File(string(path,"emission_sink.csv"),header=1,delim=",") |> DataFrame;

In [135]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi

emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"] .* 0.6          # national CO2 sink (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
# EF_trans = 86/1000000                          #5-LH trucks emit on average 56.6 gCO2/tkm while 5-RD emit 84.0 gCO2/tkm
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
penalty = 5; # ton/yr;

In [564]:
Omega = ones(Float64, ncty, num_omega, nproc);  # Omega[ncty,nscena,nproc]
scen_dmd = [0.001*164.98*(1.369*1e6)*2]

1-element Vector{Float64}:
 451715.24

---

In [567]:
sink_ratio = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        sink_ratio[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i])
    end
end

In [568]:
sink_ratio

30×15 Matrix{Float64}:
  1.253       23.6117       1.64897      …  0.103061     3.62339
  1.06242     20.1726       1.42541         0.084108     3.26693
  2.91722     54.2602       3.73934         0.251941     7.67758
  2.86595     52.8343       3.62224         0.252098     7.19141
 11.9687     228.882       16.2537          0.935821    37.8761
  0.103673     1.95778      0.137045     …  0.00845406   0.304346
  2.00093     37.3624       2.59412         0.16822      5.51583
  2.05865     33.5718       1.96628         0.501466     0.659824
  2.11562     39.2694       2.70491         0.183547     5.50642
  0.699213    12.7714       0.868218        0.0629623    1.65028
  0.221656     4.18854      0.293312     …  0.0181529    0.648726
  0.0695702    1.33124      0.094376        0.00541736   0.221256
  0.647481    12.4196       0.885065        0.0496054    2.10692
  ⋮                                      ⋱              
  3.4848      60.9933       3.95068         0.354071     5.59059
  1.645

---

### Solve Subproblem - Get Dual $\pi$ & $\alpha$

In [653]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
#     set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= 0.8*capacity[!, 2:end][i,k])
        end
    end

    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];
    

    ###########
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    
    
    G = 0
    g = 0
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, theta >= sum(x .* G) + g); 
        end
    end
    
    
    @objective(model, Min, sum(proD)+theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result

end

masterprob (generic function with 1 method)

In [654]:
function subprob(x_hat, s) # s represents specific scenario     
    
    model = Model(Gurobi.Optimizer)
#     set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, unmeet)

    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5] * scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11] * scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14] * scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15] * scaler[k]) for k in battery for j in 1:ncty] 
    
# #     cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
#     cstr_fmk = [@constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty for j in 1:ncty) + unmeet == scen_dmd[s])]
#     tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
#     cstr_fmko = [@constraint(model, y[i,j,mkt_proc] == 0) for i in 1:ncty for j in tmp]
    cstr_fmk = [@constraint(model, sum(y[i,j,14] for i in 1:ncty for j in 1:ncty) + unmeet == scen_dmd[s])] 
    
    
    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
        end
        transD[j] = arc_emi  # ncty*1 matrix
    end
    
    
    pro_sink = zeros(AffExpr, ncty, nproc)
    sink_ratio = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            sink_ratio[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i])
            pro_sink[i,k] = (sum(y[i,j,k] for j in 1:ncty)) * regional_EF[i, k+1] * (sink_c[i]/emission_c[i])
        end
    end
    proS = sum(pro_sink)
    
    
    overprod_penalty = sum((x_hat - sum(y, dims=2)[:,1,:]) .* sink_ratio)
    
    ############### Obj function ##############
    @objective(model, Min, sum(transD) + unmeet*penalty - proS)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_unmeet = JuMP.value.(unmeet)
    
    
    ################## get dual and calculate cuts ################
    dual_pi = zeros(ncty,nproc)

    op_pi = [getdual(con) for con in cstr_op]
    cth_pi = [getdual(con) for con in cstr_cth]
    cell_pi = [getdual(con) for con in cstr_cell]
    noncell_pi = [getdual(con) for con in cstr_noncell]
    battery_pi = [getdual(con) for con in cstr_battery]
    fmk_alp = [getdual(con) for con in cstr_fmk]


    dual_pi[:, cathode] = reshape(cth_pi, ncty, length(cathode)) .* Omega[:,s,cathode] .* repeat(scaler[cathode]', ncty, 1)
    dual_pi[:, cell] = reshape(cell_pi, ncty, length(cell)) .* Omega[:,s,cell] .* repeat(scaler[cell]', ncty, 1)
    dual_pi[:, noncell] = reshape(noncell_pi, ncty, length(noncell)) .* Omega[:,s,noncell] .* repeat(scaler[noncell]', ncty, 1)
    dual_pi[:, battery] = reshape(battery_pi, ncty, length(battery)) .* Omega[:,s,battery] .* repeat(scaler[battery]', ncty, 1)
    dual_pi += reshape(op_pi, ncty, nproc) .* Omega[:,s,:]
    
    dual_alpha = sum(fmk_alp) * scen_dmd[s]

    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "unmeet_opt"=>sub_unmeet, "pi"=>dual_pi, "alp"=>dual_alpha]);
    return result
    
end

subprob (generic function with 1 method)

In [655]:
function add_cuts(x)
    G = [] # gradient
    g = [] # intersection
    qy = 0
    
    
    for s in 1:num_omega
        res = subprob(x, s)        
        push!(G, res["pi"])   # gradient
        push!(g, res["alp"])  # intersection
        qy += res["qyhat"]
    end
    
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    
    
    z_hat = sum(proD) + qy/num_omega
    Exp_G = sum(G) / num_omega
    Exp_g = sum(g) / num_omega
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>Exp_g])
    return cut, z_hat

end

add_cuts (generic function with 1 method)

In [656]:
function main(toler)
    i = 0
    z_ub = 1e8
    x_opt = 0
    
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = add_cuts(x_hat)
        
        if z_hat < z_ub
            x_opt = x_hat
            z_ub = z_hat
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    
    result = Dict(["z_lb"=>z_lb, "z_ub"=>z_ub, "x_opt"=>x_opt, "iteration"=>i])
    return result
    
end

main (generic function with 1 method)

-----

In [657]:
M = -20000000
toler = 0.01

0.01

In [658]:
i = 0
z_ub = 1e8
x_opt = 0


cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 464 rows, 451 columns and 1290 nonzeros
Model fingerprint: 0x68cbfabe
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [2e-01, 5e+01]
  Bounds range     [2e+07, 2e+07]
  RHS range        [2e+02, 8e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 464 rows and 451 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.0000000e+07   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -2.000000000e+07

User-callback calls 31, time in user-callback 0.00 sec


-2.0e7

In [659]:
x_hat

30×15 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮

In [660]:
z_lb, z_ub

(-2.0e7, 1.0e8)

In [661]:
(z_ub - z_lb)

1.2e8

In [662]:
new_cut, z_hat = add_cuts(x_hat)
if z_hat < z_ub
    x_opt = x_hat
    z_ub = z_hat
end

push!(cuts, new_cut)
res1 = masterprob(cuts)
z_lb = res1["z_lb"]
x_hat = res1["x_hat"]

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 871 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0x54783d97
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-05, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+05, 5e+05]
Presolve removed 871 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2585762e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.258576200e+06

User-callback calls 24, time in user-callback 0.00 sec
Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, us

30×15 Matrix{Float64}:
  4960.0       0.0  0.0           0.0  …     0.0   0.0        0.0
 46959.8       0.0  0.0           0.0        0.0   0.0        0.0
  1760.0       0.0  1.1623e5  53600.0        0.0   0.0        0.0
   400.0    2480.0  0.0       65524.5        0.0   0.0        0.0
 31200.0       0.0  0.0           0.0        0.0   7.75558e5  0.0
     0.0       0.0  0.0           0.0  …     0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0  116000.0  0.0           0.0        0.0   0.0        0.0
     0.0     867.2  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0  …  3102.23  0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0       0.0  0.0           0.0        0.0   0.0  

In [640]:
z_lb, z_ub

(-9.42785727133547e6, 2.2585762e6)

In [641]:
(z_ub - z_lb) 

1.168643347133547e7

In [643]:
x_hat

30×15 Matrix{Float64}:
  4960.0      0.0      0.0      0.0  …     0.0   0.0        0.0
 20694.4      0.0      0.0      0.0        0.0   0.0        0.0
  1760.0      0.0  80432.2  53600.0        0.0   0.0        0.0
   400.0   2480.0      0.0  28835.3        0.0   0.0        0.0
 31200.0      0.0      0.0      0.0        0.0   5.36693e5  0.0
     0.0      0.0      0.0      0.0  …     0.0   0.0        0.0
     0.0      0.0      0.0      0.0        0.0   0.0        0.0
     0.0  80109.4      0.0      0.0        0.0   0.0        0.0
     0.0      0.0      0.0      0.0        0.0   0.0        0.0
     0.0      0.0      0.0      0.0        0.0   0.0        0.0
     0.0      0.0      0.0      0.0  …  2146.77  0.0        0.0
     0.0      0.0      0.0      0.0        0.0   0.0        0.0
     0.0      0.0      0.0      0.0        0.0   0.0        0.0
     ⋮                               ⋱                      
     0.0      0.0      0.0      0.0        0.0   0.0        0.0
     0.0      0.0   

In [644]:
s 

1

In [649]:
model = Model(Gurobi.Optimizer)
#     set_silent(model)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, unmeet >= 0)

cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5] * scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11] * scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14] * scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15] * scaler[k]) for k in battery for j in 1:ncty] 
cstr_fmk = [@constraint(model, sum(y[i,j,14] for i in 1:ncty for j in 1:ncty) + unmeet == scen_dmd[s])] 


transD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[j] = arc_emi  # ncty*1 matrix
end


pro_sink = zeros(AffExpr, ncty, nproc)
sink_ratio = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        sink_ratio[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i])
        pro_sink[i,k] = (sum(y[i,j,k] for j in 1:ncty)) * regional_EF[i, k+1] * (sink_c[i]/emission_c[i])
    end
end
proS = sum(pro_sink)


# overprod_penalty = sum((x_hat - sum(y, dims=2)[:,1,:]) .* sink_ratio)


@objective(model, Min, sum(transD) + unmeet*penalty - proS + overprod_penalty)
JuMP.optimize!(model)

qy_hat = JuMP.objective_value(model)
sub_y = JuMP.value.(y)
sub_unmeet = JuMP.value.(unmeet)

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 871 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0xce0462ac
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-05, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+05, 5e+05]
Presolve removed 871 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2585762e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.258576200e+06

User-callback calls 25, time in user-callback 0.00 sec


451715.24

In [652]:
sub_unmeet

451715.24